In [3]:
from importlib import import_module

from torch.utils.data import DataLoader

from train_eval import test
from utils import CustomDataset, DataConfig

import torch

In [9]:
def call(model_path, model_name='TextCNN', dataset='ship_data',
         embedding='embedding.npz'):
    model_module = import_module(f'models.{model_name}')
    model_config = model_module.Config()
    data_config = DataConfig(dataset, embedding)
    test_dataset = CustomDataset(data_config, data_class='test')
    test_loader = DataLoader(test_dataset, batch_size=model_config.batch_size)
    model = model_module.Model(model_config, data_config).to(data_config.device)
    test(data_config, model, test_loader, model_path)

In [10]:
call(model_path='./result/TextCNN_单层嵌入层.ckpt')

Test Loss:  0.19,  Test Acc: 93.92%
Precision, Recall and F1-Score...
              precision    recall  f1-score   support

        散杂货船     0.9092    0.9528    0.9305     44333
          渔船     0.9736    0.9855    0.9795     40276
        集装箱船     0.9330    0.9157    0.9243     18656
          油船     0.9562    0.8018    0.8722      6458
       液体散货船     0.9420    0.7466    0.8330      5919

    accuracy                         0.9392    115642
   macro avg     0.9428    0.8805    0.9079    115642
weighted avg     0.9398    0.9392    0.9383    115642

Confusion Matrix...
[[42240   862   950   141   140]
 [  552 39691    15    12     6]
 [ 1461    59 17084     6    46]
 [ 1080    88    32  5178    80]
 [ 1127    66   229    78  4419]]


In [5]:
import pickle as pkl
word_to_id=pkl.load(open('./mapping_data/pre_data/vocab.pkl', 'rb'))

In [6]:
word_to_id

{'772': 0,
 '调拌': 1,
 '可信度': 2,
 '炭火': 3,
 '造船业': 4,
 '默念': 5,
 '拘挛': 6,
 '时用': 7,
 '中高档': 8,
 '无论怎样': 9,
 '迅疾': 10,
 '长治': 11,
 '严肃认真': 12,
 '连云港': 13,
 '迟迟': 14,
 '角化': 15,
 '蛮': 16,
 '点检': 17,
 '橄榄球': 18,
 '石峡': 19,
 '发展前途': 20,
 '嵯': 21,
 '泰森': 22,
 '细菌性痢疾': 23,
 '游击战': 24,
 '高台': 25,
 '市政': 26,
 '极快': 27,
 '鸠山': 28,
 '1641': 29,
 '演员阵容': 30,
 '+\\\\': 31,
 '国家银行': 32,
 '外果皮': 33,
 '纤维状': 34,
 '手忙脚乱': 35,
 '游仙': 36,
 '工农红军': 37,
 '即为': 38,
 '未婚妻': 39,
 '排水沟': 40,
 '单纯': 41,
 '各向同性': 42,
 '八路军总部': 43,
 '雌雄异株': 44,
 '改革者': 45,
 '女歌手': 46,
 '探索': 47,
 '华人': 48,
 '飞逝': 49,
 '经济犯罪': 50,
 '上药': 51,
 '旅馆': 52,
 '上游': 53,
 '利家': 54,
 '詈': 55,
 '间隔时间': 56,
 '谤': 57,
 '口感': 58,
 '联合报': 59,
 '魔剑': 60,
 '碶': 61,
 '高速公路': 62,
 '圣洁': 63,
 '悠然自得': 64,
 '话筒': 65,
 '空仓': 66,
 '价值连城': 67,
 '2.14': 68,
 '丢脸': 69,
 '十杰': 70,
 '储君': 71,
 '公民投票': 72,
 '弯': 73,
 ';*.': 74,
 '洒满': 75,
 '抗疲劳': 76,
 'う': 77,
 '尺八': 78,
 '紫衣': 79,
 '宛如': 80,
 '墨竹': 81,
 '出奔': 82,
 '无线电视': 83,
 '供不应求': 84,
 '献礼': 85,
 '胶片': 8